In [1]:
import sys
import pprint

sys.setrecursionlimit(10000)  # if needed
pp = pprint.PrettyPrinter(depth=6, width=120)
# pp.pprint(large_object)

In [2]:
import re

def parse_facts(file_path):
    facts = {}
    target = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("target:"):
                target_line = line[len("target:"):].strip()
                key, op, val = parse_condition(target_line)
                target = (key, val)  # Only store key and expected value
                continue
            if '=' in line:
                key, val = line.split('=')
                try:
                    facts[key.strip()] = int(val.strip())
                except:
                    facts[key.strip()] = val.strip()
            elif 'is' in line:
                key, val = line.split('is')
                facts[key.strip()] = val.strip()
            else:
                facts[line.strip()] = True
    return facts, target

In [3]:
def parse_condition(cond):
    cond = cond.strip()
    if '>=' in cond:
        key, val = cond.split('>=')
        return (key.strip(), '>=', float(val.strip()))
    elif '<=' in cond:
        key, val = cond.split('<=')
        return (key.strip(), '<=', float(val.strip()))
    if '>' in cond:
        key, val = cond.split('>')
        return (key.strip(), '>', float(val.strip()))
    elif '<' in cond:
        key, val = cond.split('<')
        return (key.strip(), '<', float(val.strip()))
    elif '=' in cond:
        key, val = cond.split('=')
        try:
            return (key.strip(), '=', int(val.strip()))
        except:
            return (key.strip(), '=', val.strip())
    elif 'is' in cond:
        key, val = cond.split('is')
        return (key.strip(), 'is', val.strip())
    else:
        return (cond.strip(), '==', True)


In [4]:
def parse_rules(file_path):
    rules = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or 'IF' not in line or 'THEN' not in line:
                continue
            cond_part, conclusion = line.split('THEN')
            cond_part = cond_part.replace('IF', '').strip()
            conclusion = conclusion.strip()

            and_conditions = [c.strip() for c in re.split(r'\bAND\b', cond_part)]
            condition_list = []
            for cond in and_conditions:
                if 'OR' in cond:
                    or_parts = [c.strip() for c in cond.split('OR')]
                    or_group = [parse_condition(c) for c in or_parts]
                    condition_list.append(('OR', or_group))
                else:
                    condition_list.append(parse_condition(cond))
            conclusion_parsed = parse_condition(conclusion)
            rules.append((condition_list, conclusion_parsed))
    return rules


In [5]:
def evaluate_condition(facts, cond):
    key, op, val = cond
    if key not in facts:
        return False
    try:
        if op == '=' or op == 'is':
            return facts[key] == val
        elif op == '==':
            return bool(facts[key])
        elif op == '>':
            return float(facts[key]) > float(val)
        elif op == '<':
            return float(facts[key]) < float(val)
        elif op == '>=':
            return float(facts[key]) >= float(val)
        elif op == '<=':
            return float(facts[key]) <= float(val)
    except:
        return False
    return False


In [6]:
def evaluate_conditions(facts, conditions):
    for cond in conditions:
        if isinstance(cond, tuple) and cond[0] == 'OR':
            if not any(evaluate_condition(facts, c) for c in cond[1]):
                return False
        else:
            if not evaluate_condition(facts, cond):
                return False
    return True


In [7]:
def forward_chaining(facts, rules, target=None):
    changed = True
    while changed:
        changed = False
        print("Current facts:", facts)
        for conditions, conclusion in rules:
            key, op, val = conclusion
            if key in facts and facts[key] == val:
                continue
            if evaluate_conditions(facts, conditions):
                facts[key] = val
                changed = True
                if target and key == target[0] and facts[key] == target[1]:
                    print(f"\nTarget reached: {key} ")
                    return facts
    return facts


In [8]:
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [9]:
print("\n--- Forward Chaining ---")
final_facts = forward_chaining(facts, rules, target)

print("\nFinal inferred facts:")
for fact in final_facts:
    print(f"{fact} = {final_facts[fact]}")


--- Forward Chaining ---
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

Target reached: citrus_fruit 

Final inferred facts:
seeds = 0
diameter = 7
skin_smell = True
color = orange
perfumed = True
size = medium
fruit = orange
citrus_fruit = True


In [10]:
def backward_chaining(facts, rules, target=None):
    if target is None:
        return facts  # Nothing to prove
    
    print("\nStarting backward chaining for target:", target)
    print("Initial facts:", facts)
    
    # Create a stack of goals to prove (start with the target)
    goals_stack = [target]
    # Keep track of which goals we've already attempted to prove
    attempted_goals = set()
    
    # Continue until we've either proven the target or exhausted all possibilities
    while goals_stack:
        current_goal = goals_stack.pop()
        goal_key, goal_val = current_goal
        
        # Skip if we've already tried this goal
        goal_str = f"{goal_key}_{goal_val}"
        if goal_str in attempted_goals:
            continue
        attempted_goals.add(goal_str)
        
        print(f"\nAttempting to prove: {goal_key} = {goal_val}")
        
        # Check if this goal is already satisfied in our facts
        if goal_key in facts:
            if facts[goal_key] == goal_val:
                print(f"Already known: {goal_key} = {goal_val}")
                
                # If this was our original target, we're done!
                if current_goal == target:
                    print(f"\nTarget reached: {goal_key}")
                    return facts
                continue
            
            # For comparison operators, check if the fact satisfies the condition
            if evaluate_condition(facts, (goal_key, '=', goal_val)):
                print(f"Condition satisfied by existing fact: {goal_key} = {facts[goal_key]}")
                continue
        
        # Look for rules that could help us prove this goal
        applicable_rules = []
        for conditions, conclusion in rules:
            concl_key, concl_op, concl_val = conclusion
            if concl_key == goal_key and concl_val == goal_val:
                applicable_rules.append((conditions, conclusion))
        
        if not applicable_rules:
            print(f"No rules found that can prove: {goal_key} = {goal_val}")
            continue
        
        # Try each applicable rule
        for conditions, conclusion in applicable_rules:
            print(f"Checking rule: IF {conditions} THEN {conclusion}")
            
            # Check which conditions are not yet met
            unmet_conditions = []
            for cond in conditions:
                if isinstance(cond, tuple) and cond[0] == 'OR':
                    # For OR groups, check if any condition is already met
                    or_group = cond[1]
                    or_satisfied = False
                    unmet_or_conditions = []
                    
                    for or_cond in or_group:
                        if evaluate_condition(facts, or_cond):
                            or_satisfied = True
                            break
                        else:
                            unmet_or_conditions.append(or_cond)
                    
                    if not or_satisfied:
                        # If no OR condition is satisfied, add all to our goals
                        for or_cond in unmet_or_conditions:
                            cond_key, cond_op, cond_val = or_cond
                            goals_stack.append((cond_key, cond_val))
                        unmet_conditions.append(cond)
                else:
                    # For regular conditions
                    if not evaluate_condition(facts, cond):
                        cond_key, cond_op, cond_val = cond
                        unmet_conditions.append(cond)
                        goals_stack.append((cond_key, cond_val))
            
            # If all conditions are met, add the conclusion to our facts
            if not unmet_conditions:
                concl_key, concl_op, concl_val = conclusion
                print(f"Rule conditions satisfied, inferring: {concl_key} = {concl_val}")
                facts[concl_key] = concl_val
                
                # Check if this was our target
                if concl_key == target[0] and concl_val == target[1]:
                    print(f"\nTarget reached: {concl_key}")
                    return facts
    
    # If we've exhausted the stack without proving the target
    print(f"\nTarget could not be proven: {target[0]} = {target[1]}")
    return facts

In [11]:
def backward_chaining_improved(facts, rules, target, visited=None, depth=0):
    if visited is None:
        visited = set()

    indent = "  " * depth  # Indentation for visualizing recursion
    key, val = target

    print(f"{indent}Trying to prove: {key} = {val}")

    # First, check if the fact is already directly known
    if key in facts and facts[key] == val:
        print(f"{indent}Already known: {key} = {val}")
        return True
    
    # For comparison operations, use evaluate_condition to check if already satisfied
    if key in facts:
        for op in ['>', '<', '>=', '<=', '=', 'is', '==']:
            if evaluate_condition(facts, (key, op, val)):
                print(f"{indent}Condition satisfied: {key} {op} {val} with current value {facts[key]}")
                return True

    # Avoid loops
    target_str = f"{key}_{val}"
    if target_str in visited:
        print(f"{indent}Already visited: {key} = {val}, skipping to avoid loop.")
        return False

    visited.add(target_str)

    # Check all rules that could derive this target
    for conditions, conclusion in rules:
        concl_key, concl_op, concl_val = conclusion
        
        # Check if this rule's conclusion matches our target
        if concl_key == key and concl_val == val:
            print(f"{indent}Checking rule: IF {conditions} THEN {conclusion}")
            condition_met = True

            # Check each condition in the rule
            for cond in conditions:
                if isinstance(cond, tuple) and cond[0] == 'OR':
                    or_group = cond[1]
                    print(f"{indent}Trying OR group: {or_group}")
                    or_satisfied = False
                    
                    for or_cond in or_group:
                        cond_key, cond_op, cond_val = or_cond
                        # Check if condition is already satisfied
                        if evaluate_condition(facts, or_cond):
                            print(f"{indent}  OR condition already satisfied: {or_cond}")
                            or_satisfied = True
                            break
                        # Otherwise try to prove it
                        elif backward_chaining_improved(facts, rules, (cond_key, cond_val), visited, depth + 1):
                            or_satisfied = True
                            break
                    
                    if not or_satisfied:
                        print(f"{indent}OR group failed.")
                        condition_met = False
                        break
                else:
                    cond_key, cond_op, cond_val = cond
                    print(f"{indent}Trying condition: {cond}")
                    
                    # Check if the condition is already satisfied with current facts
                    if evaluate_condition(facts, cond):
                        print(f"{indent}  Condition already satisfied: {cond}")
                        continue
                    
                    # Try to prove this condition
                    if not backward_chaining_improved(facts, rules, (cond_key, cond_val), visited, depth + 1):
                        print(f"{indent}Condition {cond} failed.")
                        condition_met = False
                        break

            if condition_met:
                # If all conditions are met, add this fact
                facts[key] = val
                print(f"{indent}Rule succeeded, inferred: {key} = {val}")
                return True

    print(f"{indent}No rule could infer: {key} = {val}")
    return False


In [12]:
facts_back , target_back = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules_back = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [13]:
print("\n--- Backward Chaining ---")
if backward_chaining(facts_back, rules_back, target_back):
    pp.pprint(f"Target {target_back[0]} was proven.")
else:
    pp.pprint(f"Target {target_back[0]} could not be proven.")


--- Backward Chaining ---

Starting backward chaining for target: ('citrus_fruit', True)
Initial facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}

Attempting to prove: citrus_fruit = True
Checking rule: IF [('OR', [('fruit', 'is', 'lemon'), ('fruit', 'is', 'orange'), ('fruit', 'is', 'pomelo'), ('fruit', 'is', 'grapefruit')])] THEN ('citrus_fruit', '==', True)

Attempting to prove: fruit = grapefruit
Checking rule: IF [('size', 'is', 'big'), ('perfumed', '==', True), ('color', 'is', 'orange'), ('citrus_fruit', '==', True)] THEN ('fruit', 'is', 'grapefruit')

Attempting to prove: perfumed = True
Checking rule: IF [('skin_smell', '==', True)] THEN ('perfumed', '==', True)
Rule conditions satisfied, inferring: perfumed = True

Attempting to prove: size = big
Checking rule: IF [('diameter', '>', 10.0)] THEN ('size', 'is', 'big')

Attempting to prove: diameter = 10.0
No rules found that can prove: diameter = 10.0

Attempting to prove: fruit = pomelo
No rules found t

In [14]:
facts_back , target_back = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules_back = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [15]:
print("\n--- Improved Backward Chaining ---")
facts_back, target_back = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules_back = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

print("Initial facts:", facts_back)
print("Target:", target_back)

result = backward_chaining_improved(facts_back, rules_back, target_back)
if result:
    print(f"\nTarget {target_back[0]} = {target_back[1]} was proven!")
    print("\nFinal facts:")
    for fact in sorted(facts_back.keys()):
        print(f"{fact} = {facts_back[fact]}")
else:
    print(f"\nTarget {target_back[0]} = {target_back[1]} could not be proven.")
    



--- Improved Backward Chaining ---
Initial facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Target: ('citrus_fruit', True)
Trying to prove: citrus_fruit = True
Checking rule: IF [('OR', [('fruit', 'is', 'lemon'), ('fruit', 'is', 'orange'), ('fruit', 'is', 'pomelo'), ('fruit', 'is', 'grapefruit')])] THEN ('citrus_fruit', '==', True)
Trying OR group: [('fruit', 'is', 'lemon'), ('fruit', 'is', 'orange'), ('fruit', 'is', 'pomelo'), ('fruit', 'is', 'grapefruit')]
  Trying to prove: fruit = lemon
  Checking rule: IF [('size', 'is', 'medium'), ('color', 'is', 'yellow'), ('perfumed', '==', True)] THEN ('fruit', 'is', 'lemon')
  Trying condition: ('size', 'is', 'medium')
    Trying to prove: size = medium
    Checking rule: IF [('diameter', '>', 2.0), ('diameter', '<', 10.0)] THEN ('size', 'is', 'medium')
    Trying condition: ('diameter', '>', 2.0)
      Condition already satisfied: ('diameter', '>', 2.0)
    Trying condition: ('diameter', '<', 10.0)
      Condition a